In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
from functools import partial
from sqlalchemy import MetaData, Table, create_engine
from sqlalchemy.sql import text

from config import db_connection_string
from utils import tx_count_between_accounts, tx_count_between_grants

In [2]:
engine = create_engine(db_connection_string)

In [42]:
df = pd.read_sql_query("""
    SELECT 
        `t1`.`grant`,
        `t1`.`account`,
        `t1`.`id`,
        `t3`.`layer`, `t3`.`tx`,
        `t3`.`symbol`, `t3`.`amount`,
        `t4`.`symbol`, `t4`.`amount`
    FROM
        `contribution` AS `t1`
    INNER JOIN
        `grant` AS `t2` 
        ON 
            `t2`.`id` = `t1`.`grant`
    INNER JOIN
        `tx` AS `t3` 
        ON 
            `t3`.`to` = `t2`.`address`
            AND `t3`.`symbol` = `t1`.`symbol`
            AND `t3`.`amount` = `t1`.`amount_to_grant`
            AND ABS(TIMESTAMPDIFF(MINUTE, `t3`.`created_on`, `t1`.`created_on`)) <= 5
    INNER JOIN
        `tx` AS `t4`
        ON
            `t4`.`from`=`t3`.`to` 
            AND `t4`.`to`=`t3`.`from`
            AND `t4`.`created_on`>`t3`.`created_on`
""", engine)
df['id'].nunique()


942

In [43]:
engine.execute("SELECT count(*) FROM `contribution`").fetchone()

(263848,)

In [45]:
engine.execute("SELECT count(*) FROM `contribution` WHERE `grant`!='86'").fetchone()


(228370,)

In [48]:
engine.execute("SELECT count(DISTINCT `account`) FROM `contribution`").fetchone()


(21136,)

In [49]:
engine.execute("SELECT count(DISTINCT `account`) FROM `contribution`  WHERE `grant`!='86'").fetchone()


(20654,)

In [50]:
engine.execute("SELECT count(*) FROM `grant`").fetchone()


(1751,)